In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.model_selection import train_test_split
import time
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from scipy.sparse import csr_matrix
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from IPython.display import display
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [ ]:
def dcd_base(X, y, alpha, w, C, tol, it):
    D = 0
    l = len(y)

    for i in range(l):
        xi, yi = X[i], y[i]
        G = yi * np.dot(w, xi) - 1

        if alpha[i] == 0 and G >= 0:
            continue
        elif 0 < alpha[i] < C and abs(G) < tol:
            continue

        Qii = np.dot(xi, xi) + D
        alpha_i_old = alpha[i]
        alpha[i] -= G / Qii
        alpha[i] = min(max(alpha[i], 0), C)
        w += (alpha[i] - alpha_i_old) * yi * xi

    return w, alpha

def dcd_random_perm(X, y, alpha, w, C, tol, it):
    D = 0
    l = len(y)

    for i in np.random.permutation(l):
        xi, yi = X[i], y[i]
        G = yi * np.dot(w, xi) - 1

        if alpha[i] == 0 and G >= 0:
            continue
        elif 0 < alpha[i] < C and abs(G) < tol:
            continue

        Qii = np.dot(xi, xi) + D
        alpha_i_old = alpha[i]
        alpha[i] -= G / Qii
        alpha[i] = min(max(alpha[i], 0), C)
        w += (alpha[i] - alpha_i_old) * yi * xi

    return w, alpha

def dcd_online(X, y, alpha, w, C, tol, it):
    D = 0
    l = len(y)

    i = np.random.randint(0, l)
    xi, yi = X[i], y[i]
    G = yi * np.dot(w, xi) - 1

    if alpha[i] == 0 and G >= 0:
        return w, alpha
    elif 0 < alpha[i] < C and abs(G) < tol:
        return w, alpha

    Qii = np.dot(xi, xi) + D
    alpha_i_old = alpha[i]
    alpha[i] -= G / Qii
    alpha[i] = min(max(alpha[i], 0), C)
    w += (alpha[i] - alpha_i_old) * yi * xi

    return w, alpha

def dcd_with_shrinking(X, y, alpha, w, C, tol, it):
    D = 0
    l, n = X.shape

    if it == 0:
        dcd_with_shrinking.active_set = np.ones(l, dtype=bool)
        dcd_with_shrinking.shrink_counter = 0

    active_set = dcd_with_shrinking.active_set

    G_proj = compute_projected_gradient(alpha, X, y, w, D, C)

    bar_G = G_proj[active_set].max()
    under_G = G_proj[active_set].min()

    for i in np.random.permutation(np.where(active_set)[0]):
        xi, yi = X[i], y[i]
        G = yi * np.dot(w, xi) - 1 

        if alpha[i] == 0 and G >= 0:
            continue
        elif 0 < alpha[i] < C and abs(G) < tol:
            continue

        Qii = np.dot(xi, xi) + D
        alpha_old_i = alpha[i]
        alpha[i] -= G / Qii
        alpha[i] = min(max(alpha[i], 0), C)
        w += (alpha[i] - alpha_old_i) * yi * xi

    for i in range(l):
        if not active_set[i]:
            continue
        if alpha[i] == 0 and G_proj[i] > bar_G:
            active_set[i] = False
        elif alpha[i] == C and G_proj[i] < under_G:
            active_set[i] = False

    dcd_with_shrinking.shrink_counter += 1
    if dcd_with_shrinking.shrink_counter % 10 == 0 or np.sum(active_set) < l * 0.1:
        active_set[:] = True

    return w, alpha

methods_dense = {
    "Base": dcd_base,
    "Random Permutation": dcd_random_perm,
    "Online": dcd_online,
    "With Shrinking": dcd_with_shrinking
}

def compute_projected_gradient(alpha, X, y, w, D, C):
    G_proj = []
    for i in range(len(alpha)):
        G = y[i] * np.dot(w, X[i]) - 1
        if alpha[i] == 0:
            G_proj.append(min(0, G))
        elif alpha[i] == C:  
            G_proj.append(max(0, G))
        else:
            G_proj.append(G)
    return np.array(G_proj)

def dual_objective(alpha, X, y, D):
    w = np.dot((alpha * y), X)
    loss = 0.5 * np.dot(w, w) - np.sum(alpha)
    return loss


def kkt_violations(alpha, X, y, w, D, C, tol=1e-3):
    violations = 0
    for i in range(len(alpha)):
        G = y[i] * np.dot(w, X[i]) - 1
        if alpha[i] == 0 and G < -tol:
            violations += 1
        elif 0 < alpha[i] < C and abs(G) > tol:
            violations += 1
        elif alpha[i] == C and G > tol:
            violations += 1
    return violations / len(alpha)


def primal_gap(w, X, y, C, fP_star):
    fP = primal_objective(w, X, y, C)
    return abs(fP - fP_star) / abs(fP_star)


def primal_objective(w, X, y, C):
    margins = 1 - y * (X @ w)
    hinge_loss = np.sum(np.maximum(0, margins))
    return np.dot(w, w) + C * hinge_loss


def evaluate_with_criteria_dense(X, y, methods, C=1.0, tol=1e-3, max_iter=1000):
    D = 0
    results = []

    for name, method in methods.items():
        for criterion_name in ['delta_alpha', 'projected_gradient', 'dual_objective']:
            alpha = np.zeros(len(y))
            w = np.zeros(X.shape[1])
            alpha_old = alpha.copy()
            f_old = dual_objective(alpha, X, y, D)
            convergence_log = [] 

            start_time = time.time()

            for it in range(max_iter):
                w, alpha = method(X, y, alpha, w, C, tol, it)

                if criterion_name == 'delta_alpha':
                    delta = np.linalg.norm(alpha - alpha_old)
                    convergence_log.append(delta)
                    if delta < tol:
                        break
                elif criterion_name == 'projected_gradient':
                    G_proj = compute_projected_gradient(alpha, X, y, w, D, C)
                    gap = G_proj.max() - G_proj.min()
                    convergence_log.append(gap)
                    if gap < tol:
                        break
                elif criterion_name == 'dual_objective':
                    f_new = dual_objective(alpha, X, y, D)
                    diff = abs(f_new - f_old)
                    convergence_log.append(diff)
                    if diff < tol:
                        break
                    f_old = f_new

                alpha_old = alpha.copy()

            elapsed = time.time() - start_time
            accuracy = accuracy_score(y, np.sign(X @ w))

            kkt_error = kkt_violations(alpha, X, y, w, D, tol)
            fP = primal_objective(w, X, y, C)
            
            results.append({
                'Method': name,
                'Criterion': criterion_name,
                'Time': elapsed,
                'Accuracy': accuracy,
                'Iterations': it + 1,
                'Log': convergence_log,
                'KKT_violation': kkt_error,
                'fP': fP 
            })

    fP_star = min(res['fP'] for res in results)
    for res in results:
        res['PrimalGap'] = abs(res['fP'] - fP_star) / abs(fP_star)

    return results
def display_results_table(results):
    df = pd.DataFrame(results)
    display(df)

def plot_convergence(results):
    plt.figure(figsize=(12, 6))
    prop_cycle = plt.rcParams['axes.prop_cycle']
    colors = prop_cycle.by_key()['color']
    extended_colors = plt.get_cmap('tab20').colors  

    for idx, res in enumerate(results):
        label = f"{res['Method']} - {res['Criterion']}"
        iters = list(range(1, len(res['Log']) + 1))
        color = extended_colors[idx % len(extended_colors)]  
        plt.plot(iters, res['Log'], label=label, color=color, linewidth=2)

    plt.yscale('log')
    plt.xlabel("Iterations")
    plt.ylabel("Stopping criterion (log)")
    plt.title("Convergence of stopping criteria")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
def compute_projected_gradient_s(alpha, X, y, w, D, C):
    G = y * X.dot(w) - 1

    G_proj = G.copy()

    mask_lower = (alpha == 0)
    mask_upper = (alpha == C)
    mask_middle = (alpha > 0) & (alpha < C)

    G_proj[mask_lower] = np.minimum(0, G[mask_lower])
    G_proj[mask_upper] = np.maximum(0, G[mask_upper])
    G_proj[~(mask_lower | mask_upper | mask_middle)] = 0  

    return G_proj

def dual_objective_s(alpha, X, y, D):
    w = X.T.dot(alpha * y)  

    w_norm_sq = np.dot(w, w)

    loss = w_norm_sq - np.sum(alpha)

    return loss


def dcd_base_s(X, y, alpha, w, C, tol, it):
    D = 0
    l = X.shape[0]

    for i in range(l):
        xi = X.getrow(i)         
        yi = y[i]
        G = yi * xi.dot(w).item() - 1  

        if alpha[i] == 0 and G >= 0:
            continue
        elif 0 < alpha[i] < C and abs(G) < tol:
            continue

        Qii = xi.multiply(xi).sum() + D
        alpha_i_old = alpha[i]
        alpha[i] -= G / Qii
        alpha[i] = min(max(alpha[i], 0), C)

        delta_alpha = alpha[i] - alpha_i_old
        if delta_alpha != 0:
            w[xi.indices] += delta_alpha * yi * xi.data

    return w, alpha

def dcd_random_perm_s(X, y, alpha, w, C, tol, it):
    D = 0
    l = X.shape[0]

    for i in np.random.permutation(l):
        xi = X.getrow(i)  
        yi = y[i]
        G = yi * xi.dot(w).item() - 1

        if alpha[i] == 0 and G >= 0:
            continue
        elif 0 < alpha[i] < C and abs(G) < tol:
            continue

        Qii = xi.multiply(xi).sum() + D
        alpha_i_old = alpha[i]
        alpha[i] -= G / Qii
        alpha[i] = min(max(alpha[i], 0), C)

        delta_alpha = alpha[i] - alpha_i_old
        if delta_alpha != 0:
            w[xi.indices] += delta_alpha * yi * xi.data

    return w, alpha

def dcd_online_s(X, y, alpha, w, C, tol, it):
    D = 0
    l = X.shape[0]

    i = np.random.randint(0, l)
    xi = X.getrow(i)  
    yi = y[i]

    G = yi * xi.dot(w).item() - 1

    if alpha[i] == 0 and G >= 0:
        return w, alpha
    elif 0 < alpha[i] < C and abs(G) < tol:
        return w, alpha

    Qii = xi.multiply(xi).sum() + D
    alpha_i_old = alpha[i]
    alpha[i] -= G / Qii
    alpha[i] = min(max(alpha[i], 0), C)

    delta_alpha = alpha[i] - alpha_i_old
    if delta_alpha != 0:
        w[xi.indices] += delta_alpha * yi * xi.data

    return w, alpha

def dcd_with_shrinking_s(X, y, alpha, w, C, tol, it):
    D = 0
    l = X.shape[0]

    if it == 0:
        dcd_with_shrinking_s.active_set = np.ones(l, dtype=bool)
        dcd_with_shrinking_s.shrink_counter = 0

    active_set = dcd_with_shrinking_s.active_set

    G_proj = compute_projected_gradient_s(alpha, X, y, w, D, C)

    bar_G = G_proj[active_set].max()
    under_G = G_proj[active_set].min()

    for i in np.random.permutation(np.where(active_set)[0]):
        xi = X.getrow(i)
        yi = y[i]
        G = yi * xi.dot(w).item() - 1

        if alpha[i] == 0 and G >= 0:
            continue
        elif 0 < alpha[i] < C and abs(G) < tol:
            continue

        Qii = xi.multiply(xi).sum() + D
        alpha_old = alpha[i]
        alpha[i] -= G / Qii
        alpha[i] = min(max(alpha[i], 0), C)

        delta_alpha = alpha[i] - alpha_old
        if delta_alpha != 0:
            w[xi.indices] += delta_alpha * yi * xi.data

    for i in range(l):
        if not active_set[i]:
            continue
        if alpha[i] == 0 and G_proj[i] > bar_G:
            active_set[i] = False
        elif alpha[i] == C and G_proj[i] < under_G:
            active_set[i] = False

    dcd_with_shrinking_s.shrink_counter += 1
    if dcd_with_shrinking_s.shrink_counter % 10 == 0 or np.sum(active_set) < l * 0.1:
        active_set[:] = True

    return w, alpha

def generate_synthetic_dataset(n_samples, n_features, type):
    X, y = make_classification(
        n_samples=n_samples,
        n_features=n_features,
        n_informative=50,
        n_redundant=0,
        n_classes=2,
        random_state=42
    )

    density = None

    if type == 'sparse':
        X[np.abs(X) < 5.0] = 0
        X = csr_matrix(X)
        density = 100 * X.nnz / (X.shape[0] * X.shape[1])

    y = 2 * y - 1
    return X, y, density

def kkt_violations_s(alpha, X, y, w, D, C, tol=1e-3):
    violations = 0
    l = len(alpha)

    for i in range(l):
        xi = X.getrow(i)
        G = y[i] * xi.dot(w).item() - 1  

        if alpha[i] == 0 and G < -tol:
            violations += 1
        elif 0 < alpha[i] < C and abs(G) > tol:
            violations += 1
        elif alpha[i] == C and G > tol:
            violations += 1

    return violations / l

def primal_objective_s(w, X, y, C):
    margins = 1 - y * X.dot(w)
    hinge_loss = np.sum(np.maximum(0, margins))
    return np.dot(w, w) + C * hinge_loss

def primal_gap_s(w, X, y, C, fP_star):
    fP = primal_objective_s(w, X, y, C)
    return abs(fP - fP_star) / abs(fP_star)

def evaluate_with_criteria_sparse(X, y, methods, C=1.0, tol=1e-3, max_iter=1000):
    D = 0
    results = []

    for name, method in methods.items():
        for criterion_name in ['delta_alpha', 'projected_gradient', 'dual_objective']:
            alpha = np.zeros(len(y))
            w = np.zeros(X.shape[1])
            alpha_old = alpha.copy()
            f_old = dual_objective_s(alpha, X, y, D)
            convergence_log = [] 

            start_time = time.time()

            for it in range(max_iter):
                w, alpha = method(X, y, alpha, w, C, tol, it)

                if criterion_name == 'delta_alpha':
                    delta = np.linalg.norm(alpha - alpha_old)
                    convergence_log.append(delta)
                    if delta < tol:
                        break
                elif criterion_name == 'projected_gradient':
                    G_proj = compute_projected_gradient_s(alpha, X, y, w, D, C)
                    gap = G_proj.max() - G_proj.min()
                    convergence_log.append(gap)
                    if gap < tol:
                        break
                elif criterion_name == 'dual_objective':
                    f_new = dual_objective_s(alpha, X, y, D)
                    diff = abs(f_new - f_old)
                    convergence_log.append(diff)
                    if diff < tol:
                        break
                    f_old = f_new

                alpha_old = alpha.copy()

            elapsed = time.time() - start_time
            accuracy = accuracy_score(y, np.sign(X @ w))

            kkt_error = kkt_violations_s(alpha, X, y, w, D, C, tol)
            fP = primal_objective_s(w, X, y, C)

            results.append({
                'Method': name,
                'Criterion': criterion_name,
                'Time': elapsed,
                'Accuracy': accuracy,
                'Iterations': it + 1,
                'Log': convergence_log,
                'KKT_violation': kkt_error,
                'fP': fP
            })

    fP_star = min(res['fP'] for res in results)
    for res in results:
        res['PrimalGap'] = abs(res['fP'] - fP_star) / abs(fP_star)

    return results

def display_results_table(results):
    df = pd.DataFrame(results)
    display(df)

methods_sparse = {
    "Base": dcd_base_s,
    "Random Permutation": dcd_random_perm_s,
    "Online": dcd_online_s,
    "With Shrinking": dcd_with_shrinking_s
}

In [ ]:
def generate_synthetic_dataset(n_samples, n_features, n_informative, n_redundant, n_repeated, n_classes, random_state=None, type='dense'):
    X, y = make_classification(
        n_samples=n_samples,       
        n_features=n_features,       
        n_informative=n_informative,      
        n_redundant=n_redundant,        
        n_repeated=n_repeated,         
        n_classes=n_classes,          
        n_clusters_per_class=2,
        weights=[0.5, 0.5],    
        flip_y=0.05,          
        random_state=random_state, 
        shuffle=False 
    )
    density = None

    if type == 'sparse':
        threshold = np.percentile(np.abs(X), 99)
        X[np.abs(X) < threshold] = 0
        X = csr_matrix(X)
        density = 100 * X.nnz / (X.shape[0] * X.shape[1])

    y = 2 * y - 1
    feature_types = ['informative'] * n_informative + \
                    ['redundant'] * n_redundant + \
                    ['repeated'] * n_repeated + \
                    ['noise'] * (n_features - n_informative - n_redundant - n_repeated)
    return X, y, density, feature_types



In [ ]:
X_dense, y_dense, _ = generate_synthetic_dataset(n_samples=1000, n_features=100000, type='dense')

X_sparse_raw = X_dense.copy()
threshold = np.percentile(np.abs(X_sparse_raw), 99)
X_sparse_raw[np.abs(X_sparse_raw) < threshold] = 0
X_sparse = csr_matrix(X_sparse_raw)
density = 100 * X_sparse.nnz / (X_sparse.shape[0] * X_sparse.shape[1])
y_sparse = y_dense
results_dense = evaluate_with_criteria_dense(X_dense, y_dense, methods_dense)
results_sparse = evaluate_with_criteria_sparse(X_sparse, y_sparse, methods_sparse)

display_results_table(results_dense)
display_results_table(results_sparse)

In [ ]:
X_dense, y_dense, _ = generate_synthetic_dataset(n_samples=1000, n_features=500000, type='dense')

X_sparse_raw = X_dense.copy()
threshold = np.percentile(np.abs(X_sparse_raw), 99)
X_sparse_raw[np.abs(X_sparse_raw) < threshold] = 0
X_sparse = csr_matrix(X_sparse_raw)
density = 100 * X_sparse.nnz / (X_sparse.shape[0] * X_sparse.shape[1])
y_sparse = y_dense
results_dense = evaluate_with_criteria_dense(X_dense, y_dense, methods_dense)
results_sparse = evaluate_with_criteria_sparse(X_sparse, y_sparse, methods_sparse)

display_results_table(results_dense)
display_results_table(results_sparse)

In [ ]:
X_dense, y_dense, _ = generate_synthetic_dataset(n_samples=1000, n_features=100000, type='dense')

X_sparse_raw = X_dense.copy()
threshold = np.percentile(np.abs(X_sparse_raw), 99)
X_sparse_raw[np.abs(X_sparse_raw) < threshold] = 0
X_sparse = csr_matrix(X_sparse_raw)
density = 100 * X_sparse.nnz / (X_sparse.shape[0] * X_sparse.shape[1])
y_sparse = y_dense
results_dense = evaluate_with_criteria_dense(X_dense, y_dense, methods_dense, C=0.001)
results_sparse = evaluate_with_criteria_sparse(X_sparse, y_sparse, methods_sparse, C=0.001)

display_results_table(results_dense)
display_results_table(results_sparse)

In [ ]:
X_dense, y_dense, _ = generate_synthetic_dataset(n_samples=1000, n_features=100000, type='dense')

X_sparse_raw = X_dense.copy()
threshold = np.percentile(np.abs(X_sparse_raw), 99)
X_sparse_raw[np.abs(X_sparse_raw) < threshold] = 0
X_sparse = csr_matrix(X_sparse_raw)
density = 100 * X_sparse.nnz / (X_sparse.shape[0] * X_sparse.shape[1])
y_sparse = y_dense
results_dense = evaluate_with_criteria_dense(X_dense, y_dense, methods_dense, C=100)
results_sparse = evaluate_with_criteria_sparse(X_sparse, y_sparse, methods_sparse, C=100)

display_results_table(results_dense)
display_results_table(results_sparse)

In [ ]:
X_dense, y_dense, _ = generate_synthetic_dataset(n_samples=1000, n_features=100000, type='dense')

X_sparse_raw = X_dense.copy()
threshold = np.percentile(np.abs(X_sparse_raw), 99)
X_sparse_raw[np.abs(X_sparse_raw) < threshold] = 0
X_sparse = csr_matrix(X_sparse_raw)
density = 100 * X_sparse.nnz / (X_sparse.shape[0] * X_sparse.shape[1])
y_sparse = y_dense
results_dense = evaluate_with_criteria_dense(X_dense, y_dense, methods_dense, tol=1e-4)
results_sparse = evaluate_with_criteria_sparse(X_sparse, y_sparse, methods_sparse, tol=1e-4)

display_results_table(results_dense)
display_results_table(results_sparse)

In [ ]:
X_dense, y_dense, _ = generate_synthetic_dataset(n_samples=1000, n_features=100000, type='dense')

X_sparse_raw = X_dense.copy()
threshold = np.percentile(np.abs(X_sparse_raw), 99)
X_sparse_raw[np.abs(X_sparse_raw) < threshold] = 0
X_sparse = csr_matrix(X_sparse_raw)
density = 100 * X_sparse.nnz / (X_sparse.shape[0] * X_sparse.shape[1])
y_sparse = y_dense
results_dense = evaluate_with_criteria_dense(X_dense, y_dense, methods_dense, tol=1e-6)
results_sparse = evaluate_with_criteria_sparse(X_sparse, y_sparse, methods_sparse, tol=1e-6)

display_results_table(results_dense)
display_results_table(results_sparse)

In [ ]:
X_dense, y_dense, _ = generate_synthetic_dataset(n_samples=1000, n_features=100000, type='dense')

X_sparse_raw = X_dense.copy()
threshold = np.percentile(np.abs(X_sparse_raw), 99)
X_sparse_raw[np.abs(X_sparse_raw) < threshold] = 0
X_sparse = csr_matrix(X_sparse_raw)
density = 100 * X_sparse.nnz / (X_sparse.shape[0] * X_sparse.shape[1])
y_sparse = y_dense
results_dense = evaluate_with_criteria_dense(X_dense, y_dense, methods_dense, tol=1e-8)
results_sparse = evaluate_with_criteria_sparse(X_sparse, y_sparse, methods_sparse, tol=1e-8)

display_results_table(results_dense)
display_results_table(results_sparse)

In [ ]:
data = load_breast_cancer()
X = data.data
y = data.target
y = 2 * (y - 0.5)  
X = StandardScaler().fit_transform(X)

results = evaluate_with_criteria_dense(X, y, methods_dense, max_iter=1000)

plot_convergence(results)
print(len(X))
print(len(X[0]))
display_results_table(results)

In [ ]:
categories = ['alt.atheism', 'talk.religion.misc']
data = fetch_20newsgroups(subset='train', categories=categories)

vectorizer = TfidfVectorizer(
    max_features=100000,
    stop_words='english',
    min_df=1
)
X = vectorizer.fit_transform(data.data)  

y = data.target
y = 2 * y - 1  

X = X[:1000]
y = y[:1000]

d = X.nnz / (X.shape[0] * X.shape[1])

results = evaluate_with_criteria_sparse(X, y, methods_sparse, max_iter=1000)

display_results_table(results)
plot_convergence(results)

In [ ]:
def plot_convergence(results):
    plt.figure(figsize=(12, 6))
    prop_cycle = plt.rcParams['axes.prop_cycle']
    colors = prop_cycle.by_key()['color']
    extended_colors = plt.get_cmap('tab20').colors  

    for idx, res in enumerate(results):
        label = f"{res['Method']} - {res['Criterion']}"
        iters = list(range(1, len(res['Log']) + 1))
        color = extended_colors[idx % len(extended_colors)]  
        plt.plot(iters, res['Log'], label=label, color=color, linewidth=2)

    plt.yscale('log')
    plt.xlabel("Iterations")
    plt.xlim(0, 100)
    plt.ylabel("Stopping criterion (log)")
    plt.title("Convergence of stopping criteria")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.tight_layout()
    plt.show()
plot_convergence(results)